In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv('BDHSC_SCC_2025_synth_data.csv')

In [3]:
regimen_columns = ["Base_Drug_Combo", "Comp_INI", "Comp_NNRTI", "ExtraPI", "ExtraPk_En"]
df["Regimen"] = df[regimen_columns].astype(str).agg("_".join, axis=1)
df_regimen_schedule = df.groupby("ID")["Regimen"].apply(lambda x: tuple(dict.fromkeys(x))).reset_index()
df_regimen_schedule.columns = ["ID", "Regimen Schedule"]
df = pd.merge(df, df_regimen_schedule, on='ID', how='inner')
df["cross_below_250"] = df.groupby("ID")["VL"].transform(lambda x: (x < 250).any())
df["cross_below_50"] = df.groupby("ID")["VL"].transform(lambda x: (x < 50).any())
df["CD4_above_500"] = df.groupby("ID")["CD4"].transform(lambda x: (x > 500).any())
df = df.groupby(['ID', 'Gender', 'Ethnic', 'Regimen Schedule', 'cross_below_250', 'cross_below_50', 'CD4_above_500'], as_index=False).first()
df = df[['ID', 'Gender', 'Ethnic', 'Regimen Schedule', 'cross_below_250', 'cross_below_50', 'CD4_above_500']]
df["Gender"] = df["Gender"].astype("category")
df["Ethnic"] = df["Ethnic"].astype("category")
df["Regimen Schedule"] = df["Regimen Schedule"].astype("category")
df

,ID,Gender,Ethnic,Regimen Schedule,cross_below_250,cross_below_50,CD4_above_500
0,30133543973798,2,4,"(0_1_3_5_0, 0_2_3_5_1, 2_2_3_5_1, 0_0_3_5_0, 0...",True,True,True
1,198464063523553,1,4,"(3_3_3_1_0, 2_3_3_0_1, 3_3_3_0_1)",True,True,False
2,277500844403792,2,2,"(4_3_3_4_0, 1_3_3_0_1, 0_2_3_5_1, 1_3_3_0_0)",True,True,True
3,427534741934736,1,4,"(0_3_1_5_0,)",True,True,True
4,480089982195338,1,4,"(0_3_1_5_0, 0_3_2_5_0)",True,True,True
...,...,...,...,...,...,...,...
187231,18446061799332322003,1,4,"(4_3_1_5_0, 4_3_3_4_0, 0_2_3_5_1, 0_0_3_5_0, 1...",True,True,True
187232,18446128306909613693,1,4,"(0_3_2_5_0,)",True,True,True
187233,18446294480367750440,1,4,"(4_3_3_5_0, 0_3_2_5_0, 4_3_2_5_0)",True,True,True
187234,18446335763543073043,2,4,"(3_3_3_1_0, 4_3_3_1_0, 0_3_2_5_0)",True,True,True


In [ ]:
X = df[["Gender", "Ethnic", "Regimen Schedule"]]
X = pd.get_dummies(X, columns=["Gender", "Ethnic", "Regimen Schedule"], drop_first=True)

/var/folders/bw/5y6vvgld6yv27r1hv8xx5dq80000gn/T/ipykernel_43471/1855227613.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Gender"] = df["Gender"].astype("category")
/var/folders/bw/5y6vvgld6yv27r1hv8xx5dq80000gn/T/ipykernel_43471/1855227613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Ethnic"] = df["Ethnic"].astype("category")
/var/folders/bw/5y6vvgld6yv27r1hv8xx5dq80000gn/T/ipykernel_43471/1855227613.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
# Define dependent variables
y1 = df["cross_below_250"]
y2 = df["cross_below_50"]
y3 = df["CD4_above_500"]

# Fit logistic regression models
logit1 = sm.Logit(y1, sm.add_constant(X)).fit(disp=0)
logit2 = sm.Logit(y2, sm.add_constant(X)).fit(disp=0)
logit3 = sm.Logit(y3, sm.add_constant(X)).fit(disp=0)

# Display summaries of logistic regression results
logit1_summary = logit1.summary()
logit2_summary = logit2.summary()
logit3_summary = logit3.summary()

logit1_summary, logit2_summary, logit3_summary
